In [8]:
import pandas as pd
import os

## Dizionario di quelli corretti

In [9]:
def create_dic(df):
    # Define the columns (adjust as needed)
    lemma_cols = [
            'gorman_lemma_1',
    'gorman_lemma_2',
    'gorman_lemma_3',
    'gorman_lemma_4',
    'gorman_lemma_5',
    'gorman_lemma_6',
    'gorman_lemma_7',
    'gorman_lemma_8',
    'gorman_lemma_9',
    'gorman_lemma_10',
    'gorman_lemma_11',
    'gorman_lemma_12',
    'gorman_lemma_13',
    'gorman_lemma_14',
    'gpt_lemma_1',
    'gpt_lemma_2',
    'gpt_lemma_3',
    'gpt_lemma_4',
    'gpt_lemma_5',
    'gpt_lemma_6',
    'gpt_lemma_7',
    'gpt_lemma_8',
    'gpt_lemma_9',
    'gpt_lemma_10',
    'gpt_lemma_11',
    'gpt_lemma_12',
    'gpt_lemma_13',
    'gpt_lemma_14',
            'lemma_pedalion_1', 'lemma_pedalion_2', 'lemma_pedalion_3',
            'lemma_pedalion_4', 'lemma_pedalion_5', 'lemma_pedalion_6',
            'lemma_pedalion_7', 'lemma_pedalion_8', 'lemma_pedalion_9',
            'lemma_pedalion_10', 'lemma_pedalion_11', 'lemma_pedalion_12',
            'lemma_pedalion_13', 'lemma_pedalion_14', 'lemma_pedalion_15',
            'lemma_pedalion_16', 'lemma_pedalion_17'
        ]
    postag_cols = ['postag_1',
    'postag_2',
    'postag_3',
    'postag_4',
    'postag_5',
    'postag_6',
    'postag_7',
    'postag_8',
    'postag_9',
    'postag_10',
    'postag_11',
    'postag_12',
    'postag_13',
    'postag_14',
    'postag_15',
    'postag_16',
    'postag_17',
    'postag_18',
    'postag_19']

    # Dictionary to store consolidated data per @form
    form_dict = {}
    for _, row in df.iterrows():
        form = row['@form']
        if form not in form_dict:
            form_dict[form] = {"lemmas": set(), "postags": set()}
        # Gather lemma values (avoiding missing values)
        for col in lemma_cols:
            if col in df.columns:
                value = row[col]
                if pd.notnull(value):
                    form_dict[form]["lemmas"].add(value)
        # Gather postag values similarly
        for col in postag_cols:
            if col in df.columns:
                value = row[col]
                if pd.notnull(value):
                    form_dict[form]["postags"].add(value)
    return form_dict

def merge_list_of_dicts(dict_list):
    merged = {}
    for d in dict_list:
        for form, values in d.items():
            if form in merged:
                merged[form]['lemmas'].update(values['lemmas'])
                merged[form]['postags'].update(values['postags'])
            else:
                merged[form] = {
                    'lemmas': set(values['lemmas']),
                    'postags': set(values['postags'])
                }
    return merged


In [10]:
list_dir = os.listdir('correzione lemmi')
lis = [pd.read_csv('correzione lemmi/'+i) for i in list_dir]
dic_lis = []
for df in lis:
    dic = create_dic(df)
    dic_lis.append(dic)
merged_dic = merge_list_of_dicts(dic_lis)  

## Cambio i lemmi usando quelli corretti 

In [23]:
def change_lemma(df,dic):
    count = 0
    count_pt = 0
    for idx, row in df.iterrows():
        form = row['form']
        # Check if form exists in the dictionary
        if form in dic:
            valid_lemmas = dic[form]['lemmas']
            # Check if the current lemma is not in the valid lemmas
            if row['postag1'] not in valid_lemmas:
                # Pick the first lemma (convert to list for ordering, or use next(iter(valid_lemmas)) if order doesn't matter)
                if len(list(valid_lemmas))>0:
                    new_lemma = list(valid_lemmas)[0]
                    df.at[idx, 'lemma1'] = new_lemma
                    count += 1
            valid_postags = dic[form]['postags']
            # Check if the current lemma is not in the valid lemmas
            if row['postag1'] not in valid_postags:
                # Pick the first lemma (convert to list for ordering, or use next(iter(valid_lemmas)) if order doesn't matter)
                if len(list(valid_postags))>0:
                    new_postag = list(valid_postags)[0]
                    df.at[idx, 'postag1'] = new_postag
                    count_pt += 1        
    return count,count_pt,df



In [24]:
df = pd.read_csv("lemmas_postags.csv")
print(df.columns)

Index(['form', 'lemma1', 'lemma2', 'lemma3', 'lemma4', 'lemma5', 'lemma6',
       'postag1', 'postag2', 'postag3', 'postag4', 'postag5', 'postag6',
       'postag7', 'postag8', 'postag9', 'postag10'],
      dtype='object')


C:\Users\lcolo\AppData\Local\Temp\ipykernel_19884\2883933632.py:1: DtypeWarning: Columns (4,5,6,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("lemmas_postags.csv")


In [25]:
# Diciamo che vuoi comunque un dizionario dei contatori
count_dic_lemma_1 = {}
count_dic_postag_1 = {}

# Crei un’unica chiave (ad es. 'unico')
count_dic_lemma_1['unico'] = 0
count_dic_postag_1['unico'] = 0

# Leggi il CSV unificato
df = pd.read_csv("lemmas_postags.csv")

# Applica la funzione
c_l, c_pt, df_corrected = change_lemma(df, merged_dic)

# Aggiorna i contatori
count_dic_lemma_1['unico'] += c_l
count_dic_postag_1['unico'] += c_pt

# Salva il risultato
df_corrected.to_csv("lemmas_postags_corretto.csv", index=False)

# Stampa
print("Numero di lemmi modificati:", count_dic_lemma_1['unico'])
print("Numero di postag modificati:", count_dic_postag_1['unico'])


C:\Users\lcolo\AppData\Local\Temp\ipykernel_19884\1380975325.py:10: DtypeWarning: Columns (4,5,6,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("lemmas_postags.csv")


Numero di lemmi modificati: 2937
Numero di postag modificati: 110


## Carico il dataframe con i nuovi lemmi corretti dizionario_gpt_gorman_pedalion

In [26]:
dizionario  = pd.read_csv('dizionario_gpt_gorman_pedalion_primi_corretti.csv')
dizionario = dizionario.head(2354)

In [27]:
def df_to_dict(df):
    # Identify the columns for lemmas and postags
    lemma_columns = [col for col in df.columns if col.startswith('lemma_')]
    postag_columns = [col for col in df.columns if col.startswith('postag_')]
    
    result = {}
    
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        form = row['@form']
        
        # Initialize the dictionary entry if the form is encountered for the first time
        if form not in result:
            result[form] = {'lemmas': set(), 'postags': set()}
        
        # Add non-null and non-empty lemma values from the corresponding columns
        for col in lemma_columns:
            lemma = row[col]
            if pd.notnull(lemma) and lemma != "":
                result[form]['lemmas'].add(lemma)
        
        # Add non-null and non-empty postag values from the corresponding columns
        for col in postag_columns:
            postag = row[col]
            if pd.notnull(postag) and postag != "":
                result[form]['postags'].add(postag)
                
    return result

In [28]:
dizionario_dic = df_to_dict(dizionario)

## Modifico i lemmi (il risultato finale e' in output_gorman_no_anomalies_correct)

In [29]:
import pandas as pd

# Supponendo tu abbia già definito:
# - change_lemma(df, dizionario_dic)
# - dizionario_dic (il tuo dizionario con form -> set lemmi, set postag)

# Carica il CSV da correggere
df_to_correct = pd.read_csv("lemmas_postags_corretto.csv")

# Applica la funzione di correzione
count_lemma_changes, count_postag_changes, df_corrected = change_lemma(df_to_correct, dizionario_dic)

# Mostra i conteggi delle modifiche
print("Numero di lemmi modificati:", count_lemma_changes)
print("Numero di postag modificati:", count_postag_changes)

# Salva il risultato corretto in un nuovo file
df_corrected.to_csv("lemmas_postags_corretto.csv", index=False)


C:\Users\lcolo\AppData\Local\Temp\ipykernel_19884\3442318949.py:8: DtypeWarning: Columns (4,5,6,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_to_correct = pd.read_csv("lemmas_postags_corretto.csv")


Numero di lemmi modificati: 2319
Numero di postag modificati: 52
